In [1]:
import pandas as pd
import numpy as np

# 第五題

The relationship of size and shape for painted turtles are studied by Jolicoeur & Mosimann\*. The
measurements on the carapaces of 24 female and 24 male turtles can be seen in the following table.

(10%) Test if the mean vectors of the two populations are equal, given 𝛼 = 0.05.
Hint: You may wish to consider log transformation on the observations


## load data


In [2]:
df05 = pd.read_excel('HW5.xlsx')
display(df05.head())
print(f'df.shape={df05.shape}')

,Female_Length(x1),Female_Width(x2),Female_Height(x3),Male_Length(x1),Male_Width(x2),Male_Height(x3)
0,98,81,38,93,74,37
1,103,84,38,94,78,35
2,103,86,42,96,80,35
3,105,86,42,101,84,39
4,109,88,44,102,85,38


df.shape=(24, 6)


### log transformation


In [3]:
for column in df05.columns:
    df05[column] = np.log(df05[column])

df05.head()

,Female_Length(x1),Female_Width(x2),Female_Height(x3),Male_Length(x1),Male_Width(x2),Male_Height(x3)
0,4.584967,4.394449,3.637586,4.532599,4.304065,3.610918
1,4.634729,4.430817,3.637586,4.543295,4.356709,3.555348
2,4.634729,4.454347,3.737670,4.564348,4.382027,3.555348
3,4.653960,4.454347,3.737670,4.615121,4.430817,3.663562
4,4.691348,4.477337,3.784190,4.624973,4.442651,3.637586


## Hypothesis testing

$$H_0: mean_{Female\space Length(x1)}=mean_{Male\space Length(x1)},mean_{Female\space Width(x2)}=mean_{Male\space Width(x2)},mean_{Female\space Height(x3)}=mean_{Male\space Height(x3)}$$
$$H_1: H_0\space is\space not\space true$$
$$ n_1=n_2=24, X_1: male, X_2: female$$


In [4]:
Mmean1 = (df05['Male_Length(x1)']).mean()
Mmean2 = (df05['Male_Width(x2)']).mean()
Mmean3 = (df05['Male_Height(x3)']).mean()

Femean1 = (df05['Female_Length(x1)']).mean()
Femean2 = (df05['Female_Width(x2)']).mean()
Femean3 = (df05['Female_Height(x3)']).mean()

male_mean = np.row_stack((Mmean1, Mmean2, Mmean3))
Female_mean = np.row_stack((Femean1, Femean2, Femean3))

print(f'{male_mean=}\n{Female_mean=}')

male_mean=array([[4.72544365],
       [4.47757377],
       [3.70318579]])
Female_mean=array([[4.90065934],
       [4.62290892],
       [3.94028575]])


In [5]:
Male_stack = np.column_stack(
    (df05['Male_Length(x1)'], df05['Male_Width(x2)'], df05['Male_Height(x3)']))
Female_stack = np.column_stack(
    (df05['Female_Length(x1)'], df05['Female_Width(x2)'], df05['Female_Height(x3)']))

### sample variance and covariance matrix

$$ S=1/(n-1)*X_i^T*(I-1/n*J)*X_i$$

In [10]:
n = df05.shape[0]
I = np.diag(np.ones(n), 0)
J = np.ones((n, n))

male_VC = 1/(n-1)*np.dot(np.dot(Male_stack.T, (I-1/n*J)), Male_stack)
Female_VC = 1/(n-1)*np.dot(np.dot(Female_stack.T, (I-1/n*J)), Female_stack)
display(male_VC, Female_VC)

array([[0.011072  , 0.00801914, 0.00815965],
       [0.00801914, 0.00641673, 0.00600527],
       [0.00815965, 0.00600527, 0.00677276]])

array([[0.02640563, 0.02011195, 0.02491758],
       [0.02011195, 0.01619045, 0.0194243 ],
       [0.02491758, 0.0194243 , 0.0249398 ]])

### spooled

$$(n_1-1)/(n_1+n_2-2)*X_1+(n_2-1)/(n_1+n_2-2)*X_2$$


In [9]:
S_pool = (n-1)/(n+n-2)*male_VC+(n-1)/(n+n-2)*Female_VC
S_pool

array([[0.01873882, 0.01406555, 0.01653862],
       [0.01406555, 0.01130359, 0.01271478],
       [0.01653862, 0.01271478, 0.01585628]])

In [12]:
Mean_Male_minus_Female = male_mean-Female_mean
Mean_Male_minus_Female

array([[-0.17521569],
       [-0.14533516],
       [-0.23709996]])

### T_squared

$$T^2=(\bar X*1-\bar X_2)^T*((1/n_1+1/n_2)*S*{Pool})*(\bar X_1-\bar X_2)$$



In [14]:
T_squared = np.dot(np.dot(Mean_Male_minus_Female.T, np.linalg.inv(
    (1/n+1/n)*S_pool)), Mean_Male_minus_Female)
print(f'{T_squared=}')

T_squared=array([[85.05200061]])


### conclusion

$$T^2=85.052 > (n1+n2-2)*p/(n1+n2-p-1) * F_{p,n1+n2-p-1} = 46*3/44 *F_{3,44} = 8.842 $$
有顯著證據顯示拒絕 H0


# 第六題


In [15]:
from statsmodels.multivariate.manova import MANOVA

### (a) Treating the cell means as individual observations, perform two MANOVAs to test for the species effect and the nutrient effect, respectively, with 𝛼 = 0.05

In [86]:
df06 = pd.DataFrame({'CM560': [10.35, 13.41, 7.78, 10.40, 17.78, 10.4],
                     'CM720': [25.93, 38.63, 25.15, 24.25, 41.45, 29.20],
                     'Species': ['SS', 'JL', 'LP', 'SS', 'JL', 'LP'],
                     'Nutrient': ['+', '+', '+', '-', '-', '-']})
df06

,CM560,CM720,Species,Nutrient
0,10.35,25.93,SS,+
1,13.41,38.63,JL,+
2,7.78,25.15,LP,+
3,10.40,24.25,SS,-
4,17.78,41.45,JL,-
5,10.40,29.20,LP,-


In [101]:
fit = MANOVA.from_formula('CM560 + CM720  ~ Species', data=df06)
print(fit.mv_test())

                  Multivariate linear model
                                                              
--------------------------------------------------------------
       Intercept         Value   Num DF Den DF F Value  Pr > F
--------------------------------------------------------------
          Wilks' lambda   0.0023 2.0000 2.0000 429.4225 0.0023
         Pillai's trace   0.9977 2.0000 2.0000 429.4225 0.0023
 Hotelling-Lawley trace 429.4225 2.0000 1.0000 214.7112 0.0482
    Roy's greatest root 429.4225 2.0000 2.0000 429.4225 0.0023
--------------------------------------------------------------
                                                              
--------------------------------------------------------------
         C(Species)        Value  Num DF Den DF F Value Pr > F
--------------------------------------------------------------
            Wilks' lambda  0.0159 4.0000 4.0000  6.9238 0.0438
           Pillai's trace  1.4034 4.0000 6.0000  3.5281 0.0824
   Hotellin

使用package: statsmodels.multivariate.manova\
MANOVA for Species: P-value=0.0438<0.05 -> 有顯著證據顯示拒絕H0

In [100]:
fit = MANOVA.from_formula('CM560 + CM720 ~ Nutrient', data=df06)
print(fit.mv_test())

                 Multivariate linear model
                                                            
------------------------------------------------------------
       Intercept         Value  Num DF Den DF F Value Pr > F
------------------------------------------------------------
          Wilks' lambda  0.0812 2.0000 3.0000 16.9782 0.0231
         Pillai's trace  0.9188 2.0000 3.0000 16.9782 0.0231
 Hotelling-Lawley trace 11.3188 2.0000 3.0000 16.9782 0.0231
    Roy's greatest root 11.3188 2.0000 3.0000 16.9782 0.0231
------------------------------------------------------------
                                                            
------------------------------------------------------------
       C(Nutrient)       Value  Num DF Den DF F Value Pr > F
------------------------------------------------------------
           Wilks' lambda 0.6114 2.0000 3.0000  0.9535 0.4780
          Pillai's trace 0.3886 2.0000 3.0000  0.9535 0.4780
  Hotelling-Lawley trace 0.6357 2.0000 3.0

MANOVA for Nutrient: P-value=0.4780>0.05 -> 無顯著證據顯示拒絕H0

### (b) Construct a two-way ANOVA for the 560CM observations and another two-way ANOVA for the 720CM observations. Are these results consistent with the MANOVA results in (a)? If not, can you explain any differences

#### 560CM

In [62]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [85]:
model1 = ols(formula='CM560~C(Species)+C(Nutrient)', data=df06).fit()
result1 = sm.stats.anova_lm(model1)
print(result1)

              df     sum_sq    mean_sq          F    PR(>F)
C(Species)   2.0  47.476433  23.738217  10.055087  0.090456
C(Nutrient)  1.0   8.260267   8.260267   3.498902  0.202321
Residual     2.0   4.721633   2.360817        NaN       NaN


#### 720CM

In [80]:
model1 = ols(formula='CM720~C(Species)+C(Nutrient)', data=df06).fit()
result1 = sm.stats.anova_lm(model1)
print(result1)

              df      sum_sq     mean_sq          F    PR(>F)
C(Species)   2.0  262.238633  131.119317  28.819649  0.033535
C(Nutrient)  1.0    4.489350    4.489350   0.986746  0.425217
Residual     2.0    9.099300    4.549650        NaN       NaN


- 在two-way anova對於"560CM"，"Species"以及"Nutrient"的p-value各自為0.0904, 0.202皆大於0.05，故這兩個因子對560CM無顯著影響
- 在two-way anova對於"720CM"，"Species"以及"Nutrient"的p-value各自為0.0335, 0.425，僅"Species"之p-value<0.05，故僅"Species"對720CM有顯著影響

在MANOVA中
- for "Species": p-value=0.0438<0.05，表"560CM"及"720CM"皆對"Species"有顯著影響
- for "Nutrient": p-value=0.4780>0.05，表非"560CM"及720CM"皆對"Nutrient"有顯著影響

造成這個結論的可能性應為Manova會檢測交互因素的綜合影響，然而ANOVA不會捕捉到這種影響